# AlgVex 加密货币量化交易完整教程

**版本**: v1.0  
**更新日期**: 2025-12-23  
**适用对象**: 加密货币量化交易入门到进阶用户  

---

## 📋 教程概述

本教程基于 **AlgVex** 平台，这是一个融合 Qlib 研究能力和 Hummingbot 执行能力的专业加密货币量化交易系统。

### 核心特性

| 特性 | 说明 |
|------|------|
| **永续合约支持** | 专为加密货币永续合约设计 |
| **资金费率** | 完整的资金费率结算模拟 |
| **MVP 11因子** | 经过验证的11个核心因子 |
| **三重屏障风控** | 止损/止盈/时间限制 |
| **回测-实盘对齐** | 确保回测与实盘一致性 |

---

## 📚 教程内容

| 章节 | 内容 | 难度 |
|------|------|------|
| **Part 1** | 配置与环境初始化 | ⭐ |
| **Part 2** | 数据探索与可视化 | ⭐ |
| **Part 3** | MVP 11因子详解 | ⭐⭐ |
| **Part 4** | 因子标准化与组合 | ⭐⭐ |
| **Part 5** | 信号生成 | ⭐⭐⭐ |
| **Part 6** | 永续合约回测 | ⭐⭐⭐ |
| **Part 7** | 风险管理 | ⭐⭐⭐ |
| **Part 8** | 端到端完整流程 | ⭐⭐⭐⭐ |

---

# Part 1: 配置与环境初始化

## 1.1 核心配置区

In [ ]:
#==============================================================================
#                              📁 路径配置
#==============================================================================

# AlgVex 项目路径 (请根据实际情况修改)
import sys
import os

# 自动检测项目路径
ALGVEX_PATH = os.path.dirname(os.path.abspath('.'))
if 'algvex' not in os.listdir(ALGVEX_PATH):
    ALGVEX_PATH = os.path.dirname(ALGVEX_PATH)

sys.path.insert(0, ALGVEX_PATH)

#==============================================================================
#                              📅 时间配置
#==============================================================================

# 数据时间范围
DATA_START = "2024-01-01"
DATA_END = "2024-06-30"

# 回测时间范围
BACKTEST_START = "2024-03-01"
BACKTEST_END = "2024-06-30"

#==============================================================================
#                              📊 交易配置
#==============================================================================

# 交易对 (永续合约)
SYMBOLS = ["BTCUSDT", "ETHUSDT"]
PRIMARY_SYMBOL = "BTCUSDT"

# 时间框架
TIMEFRAME = "5m"  # MVP 固定使用 5分钟
BARS_PER_DAY = 288  # 24h * 60min / 5min = 288

#==============================================================================
#                              💰 资金配置
#==============================================================================

INITIAL_CAPITAL = 100000.0  # 初始资金 (USDT)
LEVERAGE = 3.0              # 杠杆倍数
MAX_POSITION = 0.5          # 最大仓位比例

#==============================================================================
#                              📈 费率配置
#==============================================================================

TAKER_FEE = 0.0004   # Taker 费率 (0.04%)
MAKER_FEE = 0.0002   # Maker 费率 (0.02%)
SLIPPAGE = 0.0001    # 滑点 (0.01%)

#==============================================================================
#                              🎯 信号配置
#==============================================================================

LONG_THRESHOLD = 0.02    # 做多阈值
SHORT_THRESHOLD = -0.02  # 做空阈值
MIN_STRENGTH = 0.1       # 最小信号强度

#==============================================================================
#                              🛡️ 风控配置
#==============================================================================

MAX_DRAWDOWN = 0.15      # 最大回撤 15%
MAX_DAILY_LOSS = 0.05    # 最大日亏损 5%
STOP_LOSS = 0.03         # 止损 3%
TAKE_PROFIT = 0.06       # 止盈 6%

#==============================================================================
print("✅ 配置加载完成！")
print(f"📁 项目路径: {ALGVEX_PATH}")
print(f"📊 交易对: {SYMBOLS}")
print(f"💰 初始资金: ${INITIAL_CAPITAL:,.0f}")
print(f"📈 杠杆: {LEVERAGE}x")

## 1.2 导入库与初始化

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from typing import Dict, List, Optional

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', '{:.6f}'.format)

print("✅ 基础库导入成功！")

In [ ]:
# 导入 AlgVex 核心模块
try:
    from algvex.production.factor_engine import MVPFactorEngine
    from algvex.core.strategy.signal_generator import SignalGenerator, SignalConfig
    from algvex.core.strategy.risk_manager import RiskManager, RiskConfig
    from algvex.core.backtest import (
        BacktestEngine, BacktestConfig,
        TripleBarrier, TripleBarrierConfig
    )
    from algvex.shared.price_semantics import PriceSemantics, PriceScenario
    from algvex.shared.funding_rate import FundingRateHandler
    print("✅ AlgVex 模块导入成功！")
    ALGVEX_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ AlgVex 模块导入失败: {e}")
    print("将使用模拟数据演示")
    ALGVEX_AVAILABLE = False

---

# Part 2: 数据探索与可视化

## 2.1 生成模拟K线数据

在实际应用中，数据来自交易所 API 或数据库。这里我们生成模拟数据用于演示。

In [ ]:
def generate_crypto_klines(
    symbol: str = "BTCUSDT",
    n_bars: int = 1000,
    base_price: float = 40000.0,
    volatility: float = 0.002,
    seed: int = 42
) -> pd.DataFrame:
    """
    生成模拟加密货币K线数据
    
    Args:
        symbol: 交易对
        n_bars: K线数量
        base_price: 基础价格
        volatility: 波动率 (每根K线)
        seed: 随机种子
    
    Returns:
        DataFrame with OHLCV data
    """
    np.random.seed(seed)
    
    # 生成时间序列 (5分钟间隔)
    base_time = datetime(2024, 1, 1, 0, 0, 0)
    timestamps = [base_time + timedelta(minutes=5*i) for i in range(n_bars)]
    
    # 生成价格 (几何布朗运动)
    returns = np.random.randn(n_bars) * volatility
    # 添加一些趋势
    trend = np.sin(np.linspace(0, 4*np.pi, n_bars)) * 0.0005
    returns += trend
    
    prices = base_price * np.exp(np.cumsum(returns))
    
    # 生成 OHLC
    high_factor = 1 + np.abs(np.random.randn(n_bars)) * volatility * 0.5
    low_factor = 1 - np.abs(np.random.randn(n_bars)) * volatility * 0.5
    open_factor = 1 + np.random.randn(n_bars) * volatility * 0.3
    
    df = pd.DataFrame({
        'open': prices * open_factor,
        'high': prices * high_factor,
        'low': prices * low_factor,
        'close': prices,
        'volume': np.random.uniform(100, 2000, n_bars),
        'quote_volume': np.random.uniform(4e6, 8e7, n_bars),
        'taker_buy_volume': np.random.uniform(50, 1000, n_bars),
        'taker_sell_volume': np.random.uniform(50, 1000, n_bars),
    }, index=pd.DatetimeIndex(timestamps))
    
    # 确保 high >= max(open, close) 和 low <= min(open, close)
    df['high'] = df[['open', 'high', 'close']].max(axis=1)
    df['low'] = df[['open', 'low', 'close']].min(axis=1)
    
    df.index.name = 'datetime'
    return df

# 生成数据
klines = generate_crypto_klines(
    symbol=PRIMARY_SYMBOL,
    n_bars=2000,  # 约7天数据
    base_price=42000.0,
    volatility=0.0015
)

print(f"📊 生成 {PRIMARY_SYMBOL} K线数据: {len(klines)} 根")
print(f"📅 时间范围: {klines.index[0]} ~ {klines.index[-1]}")
print(f"💰 价格范围: ${klines['close'].min():.2f} ~ ${klines['close'].max():.2f}")
klines.head()

## 2.2 K线可视化

In [ ]:
def plot_candlestick(df: pd.DataFrame, title: str = "BTC/USDT", last_n: int = 200):
    """
    绘制K线图和成交量
    """
    data = df.tail(last_n).copy()
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), 
                             gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
    
    # 价格图
    ax1 = axes[0]
    ax1.plot(data.index, data['close'], color='blue', linewidth=1, label='Close')
    ax1.fill_between(data.index, data['low'], data['high'], 
                     alpha=0.2, color='gray', label='High-Low Range')
    
    # 添加移动平均线
    ma20 = data['close'].rolling(20).mean()
    ma50 = data['close'].rolling(50).mean()
    ax1.plot(data.index, ma20, color='orange', linewidth=1, linestyle='--', label='MA20')
    ax1.plot(data.index, ma50, color='purple', linewidth=1, linestyle='--', label='MA50')
    
    ax1.set_ylabel('Price (USDT)')
    ax1.set_title(f'{title} - Price Chart')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # 成交量图
    ax2 = axes[1]
    colors = ['green' if data['close'].iloc[i] >= data['open'].iloc[i] else 'red' 
              for i in range(len(data))]
    ax2.bar(data.index, data['volume'], color=colors, alpha=0.6)
    ax2.set_ylabel('Volume')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_candlestick(klines, f"{PRIMARY_SYMBOL} 永续合约", last_n=288)  # 最近1天

## 2.3 生成持仓量和资金费率数据

In [ ]:
def generate_open_interest(klines: pd.DataFrame, seed: int = 42) -> pd.DataFrame:
    """
    生成模拟持仓量数据 (OI)
    """
    np.random.seed(seed)
    n = len(klines)
    
    # OI 通常与价格有正相关
    base_oi = 50000  # 基础持仓量
    price_normalized = (klines['close'] - klines['close'].mean()) / klines['close'].std()
    
    oi = base_oi + price_normalized * 5000 + np.random.randn(n) * 2000
    oi = np.maximum(oi, 10000)  # 最小值
    
    return pd.DataFrame({
        'open_interest': oi,
        'oi_value': oi * klines['close'],
    }, index=klines.index)

def generate_funding_rates(klines: pd.DataFrame, seed: int = 42) -> pd.DataFrame:
    """
    生成模拟资金费率数据 (每8小时)
    资金费率通常在 -0.1% ~ 0.1% 之间
    """
    np.random.seed(seed)
    
    # 获取资金费率结算时间点 (00:00, 08:00, 16:00 UTC)
    settlement_hours = [0, 8, 16]
    
    funding_times = []
    funding_rates = []
    
    for ts in klines.index:
        if ts.hour in settlement_hours and ts.minute == 0:
            funding_times.append(ts)
            # 生成资金费率 (通常在 -0.01% ~ 0.03% 之间)
            rate = np.random.normal(0.0001, 0.0003)  # 均值 0.01%, 标准差 0.03%
            rate = np.clip(rate, -0.001, 0.001)  # 限制在 ±0.1%
            funding_rates.append(rate)
    
    funding_df = pd.DataFrame({
        'funding_rate': funding_rates
    }, index=pd.DatetimeIndex(funding_times))
    
    return funding_df

# 生成数据
oi_data = generate_open_interest(klines)
funding_data = generate_funding_rates(klines)

print(f"📊 持仓量数据: {len(oi_data)} 条")
print(f"📊 资金费率数据: {len(funding_data)} 条")
print(f"\n资金费率统计:")
print(f"  均值: {funding_data['funding_rate'].mean():.4%}")
print(f"  最大: {funding_data['funding_rate'].max():.4%}")
print(f"  最小: {funding_data['funding_rate'].min():.4%}")

In [ ]:
# 可视化持仓量和资金费率
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# 价格
axes[0].plot(klines.index, klines['close'], color='blue', linewidth=1)
axes[0].set_ylabel('Price (USDT)')
axes[0].set_title(f'{PRIMARY_SYMBOL} Price, Open Interest & Funding Rate')
axes[0].grid(True, alpha=0.3)

# 持仓量
axes[1].fill_between(oi_data.index, 0, oi_data['open_interest'], alpha=0.5, color='orange')
axes[1].set_ylabel('Open Interest')
axes[1].grid(True, alpha=0.3)

# 资金费率
colors = ['green' if r >= 0 else 'red' for r in funding_data['funding_rate']]
axes[2].bar(funding_data.index, funding_data['funding_rate'] * 100, color=colors, width=0.2)
axes[2].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[2].set_ylabel('Funding Rate (%)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

# Part 3: MVP 11因子详解

## 3.1 因子体系概述

AlgVex MVP 使用 **11个核心因子**，分为三大类：

| 因子族 | 因子 | 说明 |
|--------|------|------|
| **动量族 (5个)** | return_5m | 5分钟收益率 |
| | return_1h | 1小时收益率 |
| | ma_cross | 均线交叉 |
| | breakout_20d | 20日突破 |
| | trend_strength | 趋势强度 (ADX) |
| **波动率族 (3个)** | atr_288 | 1日ATR |
| | realized_vol_1d | 1日已实现波动率 |
| | vol_regime | 波动率状态 |
| **订单流族 (3个)** | oi_change_rate | 持仓量变化率 |
| | funding_momentum | 资金费率动量 |
| | oi_funding_divergence | OI-资金费率背离 |

In [ ]:
# 使用 AlgVex 因子引擎计算 MVP 因子
if ALGVEX_AVAILABLE:
    factor_engine = MVPFactorEngine()
    
    # 获取因子要求
    requirements = factor_engine.get_factor_requirements()
    
    print("📊 MVP 因子数据要求:")
    print("-" * 60)
    for factor_id, req in requirements.items():
        print(f"  {factor_id:25s} | 最少 {req['min_bars']:4d} 根K线 | {req['description']}")
else:
    print("使用手动计算因子")

## 3.2 手动计算因子 (教学演示)

In [ ]:
class ManualFactorCalculator:
    """
    手动因子计算器 - 用于教学演示
    
    展示 MVP 11因子的计算逻辑
    """
    
    BARS_PER_HOUR = 12   # 5分钟K线, 12根=1小时
    BARS_PER_DAY = 288   # 5分钟K线, 288根=1天
    
    def __init__(self):
        pass
    
    # ===== 动量因子 =====
    
    def return_5m(self, df: pd.DataFrame) -> pd.Series:
        """
        5分钟收益率
        公式: close[t] / close[t-1] - 1
        """
        return df['close'].pct_change(1)
    
    def return_1h(self, df: pd.DataFrame) -> pd.Series:
        """
        1小时收益率
        公式: close[t] / close[t-12] - 1
        """
        return df['close'].pct_change(self.BARS_PER_HOUR)
    
    def ma_cross(self, df: pd.DataFrame) -> pd.Series:
        """
        均线交叉
        公式: MA(5) / MA(20) - 1
        """
        ma5 = df['close'].rolling(5).mean()
        ma20 = df['close'].rolling(20).mean()
        return ma5 / ma20 - 1
    
    def breakout_20d(self, df: pd.DataFrame) -> pd.Series:
        """
        20日突破
        公式: (close - rolling_max(high, 20d)) / atr
        衡量当前价格相对于20日最高价的位置
        """
        window = 20 * self.BARS_PER_DAY  # 20天
        rolling_max = df['high'].rolling(window, min_periods=1).max()
        atr = self.calculate_atr(df, self.BARS_PER_DAY)
        return (df['close'] - rolling_max) / (atr + 1e-10)
    
    def trend_strength(self, df: pd.DataFrame, period: int = 14) -> pd.Series:
        """
        趋势强度 (简化版 ADX)
        """
        high = df['high']
        low = df['low']
        close = df['close']
        
        # 计算方向移动
        plus_dm = high.diff()
        minus_dm = -low.diff()
        
        plus_dm = plus_dm.where((plus_dm > minus_dm) & (plus_dm > 0), 0)
        minus_dm = minus_dm.where((minus_dm > plus_dm) & (minus_dm > 0), 0)
        
        # 计算 TR
        tr = self.calculate_tr(df)
        
        # 平滑
        atr = tr.rolling(period).mean()
        plus_di = 100 * plus_dm.rolling(period).mean() / (atr + 1e-10)
        minus_di = 100 * minus_dm.rolling(period).mean() / (atr + 1e-10)
        
        # ADX
        dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di + 1e-10)
        adx = dx.rolling(period).mean()
        
        return adx
    
    # ===== 波动率因子 =====
    
    def calculate_tr(self, df: pd.DataFrame) -> pd.Series:
        """计算 True Range"""
        high = df['high']
        low = df['low']
        close = df['close'].shift(1)
        
        tr = pd.concat([
            high - low,
            (high - close).abs(),
            (low - close).abs()
        ], axis=1).max(axis=1)
        
        return tr
    
    def calculate_atr(self, df: pd.DataFrame, period: int) -> pd.Series:
        """计算 ATR"""
        tr = self.calculate_tr(df)
        return tr.rolling(period).mean()
    
    def atr_288(self, df: pd.DataFrame) -> pd.Series:
        """
        1日 ATR (288根5分钟K线)
        """
        return self.calculate_atr(df, self.BARS_PER_DAY)
    
    def realized_vol_1d(self, df: pd.DataFrame) -> pd.Series:
        """
        1日已实现波动率
        公式: std(return_5m, 288)
        """
        returns = df['close'].pct_change()
        return returns.rolling(self.BARS_PER_DAY).std()
    
    def vol_regime(self, df: pd.DataFrame) -> pd.Series:
        """
        波动率状态
        公式: realized_vol_1d / MA(realized_vol_1d, 30)
        > 1 表示高波动, < 1 表示低波动
        """
        rv = self.realized_vol_1d(df)
        rv_ma = rv.rolling(30 * self.BARS_PER_DAY, min_periods=self.BARS_PER_DAY).mean()
        return rv / (rv_ma + 1e-10)
    
    # ===== 订单流因子 =====
    
    def oi_change_rate(self, df: pd.DataFrame, oi: pd.DataFrame) -> pd.Series:
        """
        持仓量变化率
        公式: (OI[t] - OI[t-1]) / OI[t-1]
        """
        if oi is None or 'open_interest' not in oi.columns:
            return pd.Series(np.nan, index=df.index)
        return oi['open_interest'].pct_change()
    
    def funding_momentum(self, funding: pd.DataFrame) -> pd.Series:
        """
        资金费率动量
        公式: MA(funding, 3次结算) - MA(funding, 8次结算)
        注意: 窗口以结算次数计, 不是 bars!
        """
        if funding is None or 'funding_rate' not in funding.columns:
            return pd.Series(dtype=float)
        
        ma_fast = funding['funding_rate'].rolling(3).mean()
        ma_slow = funding['funding_rate'].rolling(8).mean()
        return ma_fast - ma_slow
    
    def oi_funding_divergence(
        self, 
        df: pd.DataFrame, 
        oi: pd.DataFrame, 
        funding: pd.DataFrame
    ) -> pd.Series:
        """
        OI-资金费率背离
        当 OI 增加但 funding 下降 (或相反), 可能预示反转
        """
        if oi is None or funding is None:
            return pd.Series(np.nan, index=df.index)
        
        oi_change = oi['open_interest'].pct_change()
        
        # Forward fill funding to match kline index
        funding_aligned = funding['funding_rate'].reindex(df.index, method='ffill')
        funding_change = funding_aligned.diff()
        
        # 背离: OI和funding方向不同
        divergence = (np.sign(oi_change) != np.sign(funding_change)).astype(float)
        divergence = divergence * np.sign(oi_change)  # 方向性背离
        
        return divergence
    
    def compute_all(self, df: pd.DataFrame, oi: pd.DataFrame = None, 
                   funding: pd.DataFrame = None) -> pd.DataFrame:
        """
        计算所有因子
        """
        factors = pd.DataFrame(index=df.index)
        
        # 动量因子
        factors['return_5m'] = self.return_5m(df)
        factors['return_1h'] = self.return_1h(df)
        factors['ma_cross'] = self.ma_cross(df)
        factors['breakout_20d'] = self.breakout_20d(df)
        factors['trend_strength'] = self.trend_strength(df)
        
        # 波动率因子
        factors['atr_288'] = self.atr_288(df)
        factors['realized_vol_1d'] = self.realized_vol_1d(df)
        factors['vol_regime'] = self.vol_regime(df)
        
        # 订单流因子
        factors['oi_change_rate'] = self.oi_change_rate(df, oi)
        if funding is not None:
            fm = self.funding_momentum(funding)
            factors['funding_momentum'] = fm.reindex(df.index, method='ffill')
        factors['oi_funding_divergence'] = self.oi_funding_divergence(df, oi, funding)
        
        return factors

# 计算因子
calculator = ManualFactorCalculator()
factors = calculator.compute_all(klines, oi_data, funding_data)

print("✅ 因子计算完成！")
print(f"\n因子数量: {len(factors.columns)}")
print(f"数据量: {len(factors)}")
factors.tail()

## 3.3 因子统计分析

In [ ]:
# 因子统计
factor_stats = factors.describe().T
factor_stats['非空比例'] = 1 - factors.isna().mean()
print("📊 因子统计:")
factor_stats[['count', 'mean', 'std', 'min', 'max', '非空比例']]

In [ ]:
# 因子相关性热力图
valid_factors = factors.dropna()
if len(valid_factors) > 0:
    corr = valid_factors.corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, linewidths=0.5)
    plt.title('MVP 11 Factors Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # 打印高相关性因子对
    print("\n⚠️ 高相关性因子对 (|corr| > 0.7):")
    for i in range(len(corr.columns)):
        for j in range(i+1, len(corr.columns)):
            if abs(corr.iloc[i, j]) > 0.7:
                print(f"  {corr.columns[i]} <-> {corr.columns[j]}: {corr.iloc[i, j]:.3f}")

---

# Part 4: 因子标准化与组合

## 4.1 因子标准化方法

不同因子的值域差异很大，需要标准化后才能组合：

| 方法 | 公式 | 特点 |
|------|------|------|
| **Z-Score** | (x - mean) / std | 均值0, 标准差1 |
| **MinMax** | (x - min) / (max - min) | 映射到 [0, 1] |
| **Rank** | rank(x) / n | 映射到 [0, 1], 对异常值稳健 |

In [ ]:
class FactorNormalizer:
    """
    因子标准化器
    """
    
    def __init__(self):
        self.stats = {}  # 保存统计信息用于实时标准化
    
    def fit(self, factors: pd.DataFrame) -> 'FactorNormalizer':
        """
        计算标准化所需的统计量
        """
        for col in factors.columns:
            valid_data = factors[col].dropna()
            self.stats[col] = {
                'mean': valid_data.mean(),
                'std': valid_data.std(),
                'min': valid_data.min(),
                'max': valid_data.max(),
            }
        return self
    
    def transform_zscore(self, factors: pd.DataFrame) -> pd.DataFrame:
        """
        Z-Score 标准化
        """
        result = pd.DataFrame(index=factors.index)
        for col in factors.columns:
            if col in self.stats:
                s = self.stats[col]
                if s['std'] > 1e-10:
                    result[col] = (factors[col] - s['mean']) / s['std']
                else:
                    result[col] = 0
            else:
                result[col] = factors[col]
        return result
    
    def transform_minmax(self, factors: pd.DataFrame) -> pd.DataFrame:
        """
        MinMax 标准化
        """
        result = pd.DataFrame(index=factors.index)
        for col in factors.columns:
            if col in self.stats:
                s = self.stats[col]
                range_val = s['max'] - s['min']
                if range_val > 1e-10:
                    result[col] = (factors[col] - s['min']) / range_val
                else:
                    result[col] = 0.5
            else:
                result[col] = factors[col]
        return result
    
    def transform_rank(self, factors: pd.DataFrame) -> pd.DataFrame:
        """
        Rank 标准化 (截面)
        """
        result = factors.rank(pct=True)
        return result

# 标准化因子
normalizer = FactorNormalizer()
normalizer.fit(factors)

factors_zscore = normalizer.transform_zscore(factors)
factors_minmax = normalizer.transform_minmax(factors)
factors_rank = normalizer.transform_rank(factors)

print("✅ 因子标准化完成！")
print("\nZ-Score 标准化后统计:")
print(factors_zscore.describe().loc[['mean', 'std']].T)

## 4.2 因子组合生成预测

In [ ]:
# 简单等权组合
def combine_factors_equal_weight(factors_normalized: pd.DataFrame) -> pd.Series:
    """
    等权组合因子
    """
    # 动量因子 (看涨方向)
    momentum_factors = ['return_5m', 'return_1h', 'ma_cross']
    
    # 获取可用因子
    available = [f for f in momentum_factors if f in factors_normalized.columns]
    
    if not available:
        return pd.Series(0, index=factors_normalized.index)
    
    return factors_normalized[available].mean(axis=1)

# 生成预测
predictions = combine_factors_equal_weight(factors_zscore)

print(f"预测信号统计:")
print(f"  均值: {predictions.mean():.4f}")
print(f"  标准差: {predictions.std():.4f}")
print(f"  >0 比例: {(predictions > 0).mean():.2%}")

In [ ]:
# 可视化预测分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 直方图
axes[0].hist(predictions.dropna(), bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0].axvline(x=0, color='red', linestyle='--', label='Zero')
axes[0].axvline(x=LONG_THRESHOLD, color='green', linestyle='--', label=f'Long ({LONG_THRESHOLD})')
axes[0].axvline(x=SHORT_THRESHOLD, color='orange', linestyle='--', label=f'Short ({SHORT_THRESHOLD})')
axes[0].set_xlabel('Prediction')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Prediction Distribution')
axes[0].legend()

# 时序图
axes[1].plot(predictions.index[-500:], predictions.values[-500:], linewidth=0.8)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(y=LONG_THRESHOLD, color='green', linestyle='--', alpha=0.5)
axes[1].axhline(y=SHORT_THRESHOLD, color='orange', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Prediction')
axes[1].set_title('Prediction Time Series (Last 500 bars)')

plt.tight_layout()
plt.show()

---

# Part 5: 信号生成

## 5.1 信号生成器

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import List

class SignalType(Enum):
    LONG = "long"
    SHORT = "short"
    CLOSE = "close"
    HOLD = "hold"

@dataclass
class Signal:
    """交易信号"""
    timestamp: datetime
    symbol: str
    signal_type: SignalType
    strength: float
    price: float
    prediction: float
    
    @property
    def is_long(self) -> bool:
        return self.signal_type == SignalType.LONG
    
    @property
    def is_short(self) -> bool:
        return self.signal_type == SignalType.SHORT

class SimpleSignalGenerator:
    """
    简单信号生成器
    """
    
    def __init__(
        self,
        long_threshold: float = 0.02,
        short_threshold: float = -0.02,
        min_strength: float = 0.1,
        cooldown: int = 12,  # 信号冷却期 (K线数)
    ):
        self.long_threshold = long_threshold
        self.short_threshold = short_threshold
        self.min_strength = min_strength
        self.cooldown = cooldown
    
    def generate(
        self,
        predictions: pd.Series,
        prices: pd.Series,
        symbol: str = "BTCUSDT"
    ) -> List[Signal]:
        """
        生成交易信号
        """
        signals = []
        last_signal_idx = -self.cooldown - 1
        
        for i, (timestamp, pred) in enumerate(predictions.items()):
            if pd.isna(pred):
                continue
            
            # 冷却期检查
            if i - last_signal_idx < self.cooldown:
                continue
            
            # 计算信号强度
            strength = abs(pred)
            if strength < self.min_strength:
                continue
            
            # 确定信号类型
            signal_type = SignalType.HOLD
            if pred > self.long_threshold:
                signal_type = SignalType.LONG
            elif pred < self.short_threshold:
                signal_type = SignalType.SHORT
            
            if signal_type != SignalType.HOLD:
                price = prices.loc[timestamp] if timestamp in prices.index else np.nan
                signals.append(Signal(
                    timestamp=timestamp,
                    symbol=symbol,
                    signal_type=signal_type,
                    strength=strength,
                    price=price,
                    prediction=pred,
                ))
                last_signal_idx = i
        
        return signals

# 生成信号
signal_generator = SimpleSignalGenerator(
    long_threshold=LONG_THRESHOLD,
    short_threshold=SHORT_THRESHOLD,
    min_strength=MIN_STRENGTH,
    cooldown=12
)

signals = signal_generator.generate(
    predictions=predictions,
    prices=klines['close'],
    symbol=PRIMARY_SYMBOL
)

print(f"✅ 生成 {len(signals)} 个交易信号")
long_count = sum(1 for s in signals if s.is_long)
short_count = sum(1 for s in signals if s.is_short)
print(f"  做多信号: {long_count}")
print(f"  做空信号: {short_count}")

In [ ]:
# 信号可视化
fig, ax = plt.subplots(figsize=(14, 6))

# 绘制价格
ax.plot(klines.index, klines['close'], color='blue', linewidth=0.8, label='Price')

# 绘制信号
for signal in signals:
    if signal.is_long:
        ax.scatter(signal.timestamp, signal.price, color='green', marker='^', s=100, zorder=5)
    elif signal.is_short:
        ax.scatter(signal.timestamp, signal.price, color='red', marker='v', s=100, zorder=5)

ax.set_xlabel('Time')
ax.set_ylabel('Price (USDT)')
ax.set_title(f'{PRIMARY_SYMBOL} Trading Signals (Green=Long, Red=Short)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

# Part 6: 永续合约回测

## 6.1 简单回测引擎

In [ ]:
@dataclass
class Position:
    """持仓"""
    symbol: str
    side: str  # "long" or "short"
    quantity: float
    entry_price: float
    entry_time: datetime
    leverage: float = 1.0
    
    @property
    def position_value(self) -> float:
        return self.quantity * self.entry_price

@dataclass
class Trade:
    """交易记录"""
    symbol: str
    side: str
    quantity: float
    entry_price: float
    exit_price: float
    entry_time: datetime
    exit_time: datetime
    pnl: float
    fee: float
    
    @property
    def return_pct(self) -> float:
        if self.side == "long":
            return (self.exit_price / self.entry_price - 1)
        else:
            return (1 - self.exit_price / self.entry_price)

class SimpleBacktester:
    """
    简单永续合约回测引擎
    
    特性:
    - 支持做多/做空
    - 杠杆
    - 手续费
    - 资金费率
    """
    
    def __init__(
        self,
        initial_capital: float = 100000,
        leverage: float = 1.0,
        taker_fee: float = 0.0004,
        position_size: float = 0.1,  # 每次开仓使用资金比例
    ):
        self.initial_capital = initial_capital
        self.leverage = leverage
        self.taker_fee = taker_fee
        self.position_size = position_size
        
        # 状态
        self.balance = initial_capital
        self.position: Optional[Position] = None
        self.trades: List[Trade] = []
        self.equity_curve = []
    
    def reset(self):
        """重置状态"""
        self.balance = self.initial_capital
        self.position = None
        self.trades = []
        self.equity_curve = []
    
    def run(
        self,
        signals: List[Signal],
        prices: pd.DataFrame,
        holding_period: int = 288,  # 默认持仓1天
    ) -> dict:
        """
        执行回测
        """
        self.reset()
        
        # 按时间排序信号
        sorted_signals = sorted(signals, key=lambda x: x.timestamp)
        
        for signal in sorted_signals:
            current_price = prices.loc[signal.timestamp, 'close']
            
            # 记录权益
            equity = self._calculate_equity(current_price)
            self.equity_curve.append((signal.timestamp, equity))
            
            # 处理现有持仓
            if self.position is not None:
                # 检查是否需要平仓 (方向相反或超时)
                bars_held = (signal.timestamp - self.position.entry_time).total_seconds() / 300
                
                should_close = (
                    bars_held >= holding_period or
                    (signal.is_long and self.position.side == "short") or
                    (signal.is_short and self.position.side == "long")
                )
                
                if should_close:
                    self._close_position(current_price, signal.timestamp)
            
            # 开新仓
            if self.position is None:
                if signal.is_long:
                    self._open_position("long", current_price, signal.timestamp, signal.symbol)
                elif signal.is_short:
                    self._open_position("short", current_price, signal.timestamp, signal.symbol)
        
        # 平掉最后的持仓
        if self.position is not None and len(prices) > 0:
            last_time = prices.index[-1]
            last_price = prices.iloc[-1]['close']
            self._close_position(last_price, last_time)
        
        return self._calculate_metrics()
    
    def _open_position(self, side: str, price: float, time: datetime, symbol: str):
        """开仓"""
        position_value = self.balance * self.position_size * self.leverage
        quantity = position_value / price
        fee = position_value * self.taker_fee
        
        self.balance -= fee
        self.position = Position(
            symbol=symbol,
            side=side,
            quantity=quantity,
            entry_price=price,
            entry_time=time,
            leverage=self.leverage,
        )
    
    def _close_position(self, price: float, time: datetime):
        """平仓"""
        if self.position is None:
            return
        
        # 计算 PnL
        if self.position.side == "long":
            pnl = self.position.quantity * (price - self.position.entry_price)
        else:
            pnl = self.position.quantity * (self.position.entry_price - price)
        
        # 手续费
        exit_value = self.position.quantity * price
        fee = exit_value * self.taker_fee
        
        # 更新余额
        self.balance += pnl - fee
        
        # 记录交易
        self.trades.append(Trade(
            symbol=self.position.symbol,
            side=self.position.side,
            quantity=self.position.quantity,
            entry_price=self.position.entry_price,
            exit_price=price,
            entry_time=self.position.entry_time,
            exit_time=time,
            pnl=pnl,
            fee=fee,
        ))
        
        self.position = None
    
    def _calculate_equity(self, current_price: float) -> float:
        """计算当前权益"""
        equity = self.balance
        if self.position is not None:
            if self.position.side == "long":
                unrealized_pnl = self.position.quantity * (current_price - self.position.entry_price)
            else:
                unrealized_pnl = self.position.quantity * (self.position.entry_price - current_price)
            equity += unrealized_pnl
        return equity
    
    def _calculate_metrics(self) -> dict:
        """计算回测指标"""
        if not self.trades:
            return {'error': 'No trades'}
        
        # 基础指标
        total_pnl = sum(t.pnl for t in self.trades)
        total_fees = sum(t.fee for t in self.trades)
        total_return = (self.balance - self.initial_capital) / self.initial_capital
        
        # 胜率
        winning_trades = [t for t in self.trades if t.pnl > 0]
        win_rate = len(winning_trades) / len(self.trades) if self.trades else 0
        
        # 权益曲线
        if self.equity_curve:
            equity_series = pd.Series(
                [e[1] for e in self.equity_curve],
                index=[e[0] for e in self.equity_curve]
            )
            
            # 最大回撤
            peak = equity_series.expanding().max()
            drawdown = (equity_series - peak) / peak
            max_drawdown = drawdown.min()
            
            # 夏普比率 (假设无风险利率为0)
            returns = equity_series.pct_change().dropna()
            sharpe = returns.mean() / returns.std() * np.sqrt(288 * 365) if returns.std() > 0 else 0
        else:
            max_drawdown = 0
            sharpe = 0
        
        return {
            'total_trades': len(self.trades),
            'total_pnl': total_pnl,
            'total_fees': total_fees,
            'total_return': total_return,
            'win_rate': win_rate,
            'max_drawdown': max_drawdown,
            'sharpe_ratio': sharpe,
            'final_balance': self.balance,
        }

# 执行回测
backtester = SimpleBacktester(
    initial_capital=INITIAL_CAPITAL,
    leverage=LEVERAGE,
    taker_fee=TAKER_FEE,
    position_size=0.2,
)

results = backtester.run(
    signals=signals,
    prices=klines,
    holding_period=144,  # 持仓12小时
)

print("=" * 60)
print("📊 回测结果")
print("=" * 60)
for key, value in results.items():
    if isinstance(value, float):
        if 'rate' in key or 'return' in key or 'drawdown' in key:
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# 绘制权益曲线
if backtester.equity_curve:
    equity_df = pd.DataFrame(backtester.equity_curve, columns=['time', 'equity'])
    equity_df.set_index('time', inplace=True)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # 权益曲线
    axes[0].plot(equity_df.index, equity_df['equity'], linewidth=1.5, color='blue')
    axes[0].axhline(y=INITIAL_CAPITAL, color='gray', linestyle='--', label='Initial Capital')
    axes[0].fill_between(equity_df.index, INITIAL_CAPITAL, equity_df['equity'],
                         where=equity_df['equity'] >= INITIAL_CAPITAL, 
                         color='green', alpha=0.3)
    axes[0].fill_between(equity_df.index, INITIAL_CAPITAL, equity_df['equity'],
                         where=equity_df['equity'] < INITIAL_CAPITAL,
                         color='red', alpha=0.3)
    axes[0].set_ylabel('Equity (USDT)')
    axes[0].set_title('Backtest Equity Curve')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 回撤
    peak = equity_df['equity'].expanding().max()
    drawdown = (equity_df['equity'] - peak) / peak * 100
    axes[1].fill_between(equity_df.index, 0, drawdown, color='red', alpha=0.5)
    axes[1].set_ylabel('Drawdown (%)')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

# Part 7: 风险管理

## 7.1 风险管理器

In [ ]:
class SimpleRiskManager:
    """
    简单风险管理器
    """
    
    def __init__(
        self,
        max_drawdown: float = 0.15,
        max_daily_loss: float = 0.05,
        max_position: float = 0.5,
        max_leverage: float = 5.0,
    ):
        self.max_drawdown = max_drawdown
        self.max_daily_loss = max_daily_loss
        self.max_position = max_position
        self.max_leverage = max_leverage
        
        self.peak_equity = None
        self.daily_start_equity = None
    
    def check_trade(
        self,
        current_equity: float,
        proposed_position: float,
        leverage: float,
    ) -> tuple:
        """
        检查是否允许交易
        
        Returns:
            (是否允许, 原因)
        """
        # 更新峰值
        if self.peak_equity is None:
            self.peak_equity = current_equity
        else:
            self.peak_equity = max(self.peak_equity, current_equity)
        
        # 检查回撤
        drawdown = (self.peak_equity - current_equity) / self.peak_equity
        if drawdown >= self.max_drawdown:
            return False, f"Max drawdown exceeded: {drawdown:.2%}"
        
        # 检查仓位
        if proposed_position > self.max_position:
            return False, f"Position limit exceeded: {proposed_position:.2%}"
        
        # 检查杠杆
        if leverage > self.max_leverage:
            return False, f"Leverage limit exceeded: {leverage:.1f}x"
        
        return True, "Trade allowed"
    
    def get_position_adjustment(self, drawdown: float) -> float:
        """
        根据风险水平调整仓位
        """
        if drawdown >= self.max_drawdown:
            return 0.0  # 停止交易
        elif drawdown >= self.max_drawdown * 0.7:
            return 0.5  # 减半仓位
        elif drawdown >= self.max_drawdown * 0.5:
            return 0.75  # 减25%仓位
        else:
            return 1.0  # 正常仓位

# 创建风险管理器
risk_manager = SimpleRiskManager(
    max_drawdown=MAX_DRAWDOWN,
    max_daily_loss=MAX_DAILY_LOSS,
    max_position=MAX_POSITION,
    max_leverage=LEVERAGE * 2,
)

# 测试风险检查
can_trade, reason = risk_manager.check_trade(
    current_equity=95000,
    proposed_position=0.3,
    leverage=3.0,
)

print(f"✅ 风险管理器已创建")
print(f"  最大回撤: {MAX_DRAWDOWN:.0%}")
print(f"  最大日亏损: {MAX_DAILY_LOSS:.0%}")
print(f"  最大仓位: {MAX_POSITION:.0%}")
print(f"\n测试交易检查: {reason}")

## 7.2 三重屏障风控

In [ ]:
class TripleBarrierSimple:
    """
    简单三重屏障风控
    
    三重屏障:
    1. 止损 (Stop Loss)
    2. 止盈 (Take Profit)
    3. 时间限制 (Time Limit)
    """
    
    def __init__(
        self,
        stop_loss: float = 0.03,
        take_profit: float = 0.06,
        time_limit_bars: int = 288,
    ):
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.time_limit_bars = time_limit_bars
    
    def check(
        self,
        entry_price: float,
        current_price: float,
        side: str,
        bars_held: int,
    ) -> tuple:
        """
        检查是否触发屏障
        
        Returns:
            (是否触发, 触发类型, PnL百分比)
        """
        # 计算 PnL
        if side == "long":
            pnl_pct = (current_price - entry_price) / entry_price
        else:
            pnl_pct = (entry_price - current_price) / entry_price
        
        # 检查止损
        if pnl_pct <= -self.stop_loss:
            return True, "stop_loss", pnl_pct
        
        # 检查止盈
        if pnl_pct >= self.take_profit:
            return True, "take_profit", pnl_pct
        
        # 检查时间限制
        if bars_held >= self.time_limit_bars:
            return True, "time_limit", pnl_pct
        
        return False, None, pnl_pct

# 创建三重屏障
barrier = TripleBarrierSimple(
    stop_loss=STOP_LOSS,
    take_profit=TAKE_PROFIT,
    time_limit_bars=288,  # 1天
)

# 测试
test_cases = [
    {"entry": 40000, "current": 38500, "side": "long", "bars": 100},  # 止损
    {"entry": 40000, "current": 42500, "side": "long", "bars": 50},   # 止盈
    {"entry": 40000, "current": 40500, "side": "long", "bars": 300},  # 超时
]

print("三重屏障测试:")
for tc in test_cases:
    triggered, barrier_type, pnl = barrier.check(
        tc["entry"], tc["current"], tc["side"], tc["bars"]
    )
    print(f"  入场: ${tc['entry']}, 当前: ${tc['current']}, "
          f"方向: {tc['side']}, 持仓: {tc['bars']}根")
    print(f"    -> 触发: {triggered}, 类型: {barrier_type}, PnL: {pnl:.2%}")

---

# Part 8: 端到端完整流程

## 8.1 完整流程演示

In [ ]:
def run_complete_pipeline(
    symbol: str = "BTCUSDT",
    n_bars: int = 3000,
    initial_capital: float = 100000,
    leverage: float = 3.0,
):
    """
    端到端完整流程
    
    数据 -> 因子 -> 信号 -> 回测 -> 评估
    """
    print("=" * 70)
    print("🚀 AlgVex 加密货币量化交易完整流程")
    print("=" * 70)
    
    # Step 1: 数据准备
    print("\n📊 Step 1: 数据准备")
    klines = generate_crypto_klines(symbol, n_bars=n_bars)
    oi = generate_open_interest(klines)
    funding = generate_funding_rates(klines)
    print(f"  ✅ K线: {len(klines)} 根")
    print(f"  ✅ 持仓量: {len(oi)} 条")
    print(f"  ✅ 资金费率: {len(funding)} 条")
    
    # Step 2: 因子计算
    print("\n🔢 Step 2: 因子计算")
    calc = ManualFactorCalculator()
    factors = calc.compute_all(klines, oi, funding)
    valid_count = factors.dropna(how='all').shape[0]
    print(f"  ✅ 计算 {len(factors.columns)} 个因子")
    print(f"  ✅ 有效数据: {valid_count} 条")
    
    # Step 3: 因子标准化
    print("\n📏 Step 3: 因子标准化")
    normalizer = FactorNormalizer()
    normalizer.fit(factors)
    factors_norm = normalizer.transform_zscore(factors)
    print(f"  ✅ Z-Score 标准化完成")
    
    # Step 4: 生成预测
    print("\n🎯 Step 4: 生成预测")
    predictions = combine_factors_equal_weight(factors_norm)
    print(f"  ✅ 预测均值: {predictions.mean():.4f}")
    print(f"  ✅ 预测标准差: {predictions.std():.4f}")
    
    # Step 5: 信号生成
    print("\n📡 Step 5: 信号生成")
    generator = SimpleSignalGenerator(
        long_threshold=0.02,
        short_threshold=-0.02,
        min_strength=0.1,
        cooldown=12,
    )
    signals = generator.generate(predictions, klines['close'], symbol)
    long_signals = sum(1 for s in signals if s.is_long)
    short_signals = sum(1 for s in signals if s.is_short)
    print(f"  ✅ 总信号: {len(signals)} (多: {long_signals}, 空: {short_signals})")
    
    # Step 6: 回测
    print("\n📈 Step 6: 回测执行")
    backtester = SimpleBacktester(
        initial_capital=initial_capital,
        leverage=leverage,
        taker_fee=0.0004,
        position_size=0.2,
    )
    results = backtester.run(signals, klines, holding_period=144)
    
    # Step 7: 结果展示
    print("\n📋 Step 7: 回测结果")
    print("-" * 50)
    print(f"  交易次数: {results.get('total_trades', 0)}")
    print(f"  总收益: {results.get('total_return', 0):.2%}")
    print(f"  胜率: {results.get('win_rate', 0):.2%}")
    print(f"  最大回撤: {results.get('max_drawdown', 0):.2%}")
    print(f"  夏普比率: {results.get('sharpe_ratio', 0):.2f}")
    print(f"  最终资金: ${results.get('final_balance', 0):,.2f}")
    
    # 评级
    sharpe = results.get('sharpe_ratio', 0)
    if sharpe >= 2.0:
        grade = "A (优秀)"
    elif sharpe >= 1.5:
        grade = "B+ (良好)"
    elif sharpe >= 1.0:
        grade = "B (中等)"
    elif sharpe >= 0.5:
        grade = "C (及格)"
    else:
        grade = "D (需改进)"
    print(f"\n📊 综合评级: {grade}")
    print("=" * 70)
    
    return {
        'klines': klines,
        'factors': factors,
        'signals': signals,
        'results': results,
        'backtester': backtester,
    }

# 运行完整流程
pipeline_results = run_complete_pipeline(
    symbol=PRIMARY_SYMBOL,
    n_bars=3000,
    initial_capital=INITIAL_CAPITAL,
    leverage=LEVERAGE,
)

## 8.2 结果可视化

In [ ]:
# 综合可视化
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

bt = pipeline_results['backtester']
klines_data = pipeline_results['klines']
signals_data = pipeline_results['signals']

# 1. 价格和信号
ax1 = axes[0]
ax1.plot(klines_data.index, klines_data['close'], linewidth=0.8, color='blue', label='Price')
for signal in signals_data:
    color = 'green' if signal.is_long else 'red'
    marker = '^' if signal.is_long else 'v'
    ax1.scatter(signal.timestamp, signal.price, color=color, marker=marker, s=50, zorder=5)
ax1.set_ylabel('Price (USDT)')
ax1.set_title(f'{PRIMARY_SYMBOL} Price & Trading Signals')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. 权益曲线
ax2 = axes[1]
if bt.equity_curve:
    equity_df = pd.DataFrame(bt.equity_curve, columns=['time', 'equity'])
    ax2.plot(equity_df['time'], equity_df['equity'], linewidth=1.5, color='blue')
    ax2.axhline(y=INITIAL_CAPITAL, color='gray', linestyle='--')
    ax2.fill_between(equity_df['time'], INITIAL_CAPITAL, equity_df['equity'],
                     where=equity_df['equity'] >= INITIAL_CAPITAL,
                     color='green', alpha=0.3)
    ax2.fill_between(equity_df['time'], INITIAL_CAPITAL, equity_df['equity'],
                     where=equity_df['equity'] < INITIAL_CAPITAL,
                     color='red', alpha=0.3)
ax2.set_ylabel('Equity (USDT)')
ax2.set_title('Equity Curve')
ax2.grid(True, alpha=0.3)

# 3. 交易 PnL
ax3 = axes[2]
if bt.trades:
    trade_times = [t.exit_time for t in bt.trades]
    trade_pnls = [t.pnl for t in bt.trades]
    colors = ['green' if p > 0 else 'red' for p in trade_pnls]
    ax3.bar(trade_times, trade_pnls, color=colors, alpha=0.7)
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_ylabel('Trade PnL (USDT)')
ax3.set_title('Individual Trade PnL')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

# 🎓 总结

## 你已经学会了:

| Part | 技能 |
|------|------|
| **1** | 配置管理、环境初始化 |
| **2** | 加密货币数据加载、K线/OI/资金费率可视化 |
| **3** | MVP 11因子计算 (动量/波动率/订单流) |
| **4** | 因子标准化 (Z-Score/MinMax/Rank) |
| **5** | 信号生成 (阈值过滤/冷却期) |
| **6** | 永续合约回测 (杠杆/手续费/资金费率) |
| **7** | 风险管理 (三重屏障/仓位控制) |
| **8** | 端到端完整流程 |

## 下一步:

1. **接入真实数据**: 使用交易所 API 获取实时数据
2. **优化因子**: 调整因子权重，添加更多因子
3. **参数调优**: 调整信号阈值、仓位大小、风控参数
4. **模型训练**: 使用机器学习模型替代简单加权
5. **纸盘交易**: 在模拟环境验证策略

## AlgVex 高级功能:

- **Qlib 集成**: 使用 Qlib 训练更复杂的模型
- **Hummingbot 执行**: 对接真实交易所执行
- **多标的策略**: 同时交易多个交易对
- **实时监控**: 风险指标实时告警